## 05. Analyze 

## Setting

In [2]:
import pandas as pd 
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from review_analyzer import *

# rc('font', family='AppleGothic') # Mac 
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [3]:
data_path = "data/naver_review_preprocessing.csv"
data = pd.read_csv(data_path, parse_dates=["date"], encoding="CP949")
data.head(5)

,reviewer,review,date,weekday
0,shooooooo,종류도 알차게 많고 다 맛있어요,2024-06-07,금요일
1,쥴리08,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,2024-05-23,목요일
2,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,2024-05-19,일요일
3,DEAN KIL,비싸지만 좋은곳,2024-05-04,토요일
4,이진욱쨩,항상 친절하시고 음식도맛있네요,2024-04-21,일요일


## 1. 프롬프트

## 1) 시도 1

In [ ]:
# 내가 원하는 출력 결과
{
    "맛": {"긍정": [], "부정": []},
    "서비스": {"긍정": [], "부정": []},
    "환경": {"긍정": [], "부정": []},
    "기타": {"긍정": [], "부정": []}
}

In [85]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional 

from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

class ActionModel(BaseModel):
    positive: List[str] = Field(description="긍정을 나타내는 키워드")
    negative: List[str] = Field(description="부정을 나타내는 키워드")

class ReviewModel(BaseModel):
    category: str = Field(description="CATEGORY 문자열")
    action: ActionModel

parser = JsonOutputParser(pydantic_object=ReviewModel)

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 CATEGORY에 따라 긍/부정에 해당하는 키워드를 분류하는 역할입니다.
- 키워드는 CATEGORY에 대한 긍/부정을 나타내는 명사만 추출하세요.
- FORMAT에 맞춰 답변하세요.

# FORMAT: {format_instructions}
# CATEGORY: {category}
# SENTENCE: {sentence}
"""

prompt = PromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser(pydantic_object=ReviewModel)
chain = prompt | model | output_parser

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

print(chain.invoke({"category":"맛", "sentence": review}))
print(chain.invoke({"category":"서비스", "sentence": review}))
print(chain.invoke({"category":"가격", "sentence": review}))

{'category': '맛', 'action': {'positive': ['친절'], 'negative': ['아쉬움']}}
{'category': '서비스', 'action': {'positive': ['친절', '융통성'], 'negative': ['아쉬움']}}
{'category': '가격', 'action': {'positive': ['친절', '음식맛', '융통성'], 'negative': ['과일류']}}


## 2) 시도 2

In [ ]:
# 내가 원하는 출력 결과
{
    "맛": {"긍정": [], "부정": []},
    "서비스": {"긍정": [], "부정": []},
    "환경": {"긍정": [], "부정": []},
    "기타": {"긍정": [], "부정": []}
}

In [81]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional 

from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

class ActionModel(BaseModel):
    positive: List[str] = Field(description="긍정을 나타내는 키워드")
    negative: List[str] = Field(description="부정을 나타내는 키워드")

class ReviewModel(BaseModel):
    category: str = Field(description="CATEGORY 문자열")
    action: ActionModel

parser = JsonOutputParser(pydantic_object=ReviewModel)

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 CATEGORY에 따라 긍/부정에 해당하는 키워드를 분류하는 역할입니다.
- 키워드는 긍/부정의 대상이며, 명사만 추출하세요.
- FORMAT에 맞춰 답변하세요.

# FORMAT: {format_instructions}
# NOUNS: {nouns}
# CATEGORY: {category}
# SENTENCE: {sentence}
"""

nouns = "직원 친절 음식 맛 대한민국 호텔 부페 넘버원 뭐 조금 입장 융통 거 일류 구색 거"
nouns_new = ", ".join([x for x in set(nouns.split()) if len(x) > 1])
print(nouns_new)

prompt = PromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser(pydantic_object=ReviewModel)
chain = prompt | model | output_parser

호텔, 조금, 부페, 넘버원, 입장, 융통, 구색, 일류, 친절, 직원, 음식, 대한민국


In [63]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positive": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "items": {"type": "string"}, "title": "Positive", "type": "array"}, "negative": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "items": {"type": "string"}, "title": "Negative", "type": "array"}}, "required": ["positive", "negative"]}
```


In [60]:
from pydantic import BaseModel, Field
from typing import List

class ActionModel(BaseModel):
    positive: List[str] = Field(description="주제에 대한 간결한 설명")
    negative: List[str] = Field(description="해시태그 형식의 키워드(2개 이상)")

mydict = {
    "positive": ["A", "B", "C"],
    "negative": ["D", "E", "F"]
}
ActionModel(**mydict)

ActionModel(positive=['A', 'B', 'C'], negative=['D', 'E', 'F'])

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)
prompt.partial()

PromptTemplate(input_variables=['sentence'], template='# INSTRUCTION\n- 당신은 긍/부정 분류기입니다.\n- 각 대상 \'만족도\', \'맛\', \'서비스\', \'가격\'에 대한 평가가 긍정적인지 부정적인지를 분류하세요.\n- 대상에 대한 평가가 없는 경우 \'-\'을 표시하세요.\n- 예시를 보고 결과를 다음과 같은 딕셔너리 형식으로 출력하세요:\n\n    "만족도": "긍정/부정/-",\n    "맛": "긍정/부정/-",\n    "서비스": "긍정/부정/-",\n    "가격": "긍정/부정/-"\n\n# SENTENCE: {sentence}\n')

In [4]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
parser.get_format_instructions()

'Return a JSON object.'

In [5]:
template = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 각 대상 '만족도', '맛', '서비스', '가격'에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 대상에 대한 평가가 없는 경우 '-'을 표시하세요.
- 예시를 보고 결과를 다음과 같은 딕셔너리 형식으로 출력하세요:

    "만족도": "긍정/부정/-",
    "맛": "긍정/부정/-",
    "서비스": "긍정/부정/-",
    "가격": "긍정/부정/-"

# SENTENCE: {sentence}
"""

In [90]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 {sentence}에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- JSON 형식으로 출력하세요.
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(category):
    return {"sentence": RunnablePassthrough() , "category": RunnablePassthrough(lambda _: category)}

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke(review)


{'taste': {'feedback': '긍정'},
 'service': {'feedback': '긍정'},
 'price': {'feedback': '긍정'}}

In [109]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- 다음 KEYS를 참고하여 JSON 형식으로 출력하세요.

# KEYS:
- category <{category}반응과 관련된 대상>
- action <반응>

# SENTENCE: {sentence}
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(text):
    return RunnablePassthrough.assign(category = lambda x: text)

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke({"sentence": review})


{'taste': {'category': '맛', 'action': '긍정'},
 'service': {'category': '직원 및 음식', 'action': '긍정'},
 'price': {'category': '가격', 'action': '-'}}

In [94]:
cat = "맛"
add_runnable = RunnablePassthrough.assign(taste = lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐', 'taste': '안뇽'}

In [101]:
cat = "맛"
add_runnable = RunnablePassthrough(lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐'}

# 2. Pydantic

In [41]:
from pydantic import BaseModel 

class User(BaseModel):
    id: int
    name: str

user = User(
    id = 1,
    name = "Hong Gildong",
)

print(user.id)
print(user.name)

1
Hong Gildong


In [43]:
from pydantic import BaseModel

class Contact(BaseModel):
    email: str 
    phone: str 

class Address(BaseModel):
    street:str
    city:str
    postal_code:str

class User(BaseModel):
    id: int
    name: str
    address: Address
    contact: Contact

user = User(
    id = 103948,
    name = "Hong Gildong",
    address = {
        "street": "종로구 청와대로 1",
        "city": "서울특별시",
        "postal_code": "03048"
    },
    contact = {
        "email": "gildong@xmail.com",
        "phone": "010-1122-3344"
    }
)

user.model_dump()


{'id': 103948,
 'name': 'Hong Gildong',
 'address': {'street': '종로구 청와대로 1', 'city': '서울특별시', 'postal_code': '03048'},
 'contact': {'email': 'gildong@xmail.com', 'phone': '010-1122-3344'}}

In [44]:
user

User(id=103948, name='Hong Gildong', address=Address(street='종로구 청와대로 1', city='서울특별시', postal_code='03048'), contact=Contact(email='gildong@xmail.com', phone='010-1122-3344'))

In [55]:
# repr
from pydantic import BaseModel, Field

class User(BaseModel):
    id: int = Field(repr=True)
    name: str = Field(repr=False)

user = User(
    id = 103948,
    name = "Hong Gildong",
)

print(user)
print(user.model_dump())
print(user.model_json_schema())


id=103948
{'id': 103948, 'name': 'Hong Gildong'}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['id', 'name'], 'title': 'User', 'type': 'object'}


In [54]:
# repr
from pydantic import BaseModel, Field

class User(BaseModel):
    id: int = Field(exclude=True)
    name: str = Field(exclude=False)

user = User(
    id = 103948,
    name = "Hong Gildong",
)

print(user)
print(user.model_dump())
print(user.model_json_schema())


id=103948 name='Hong Gildong'
{'name': 'Hong Gildong'}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['id', 'name'], 'title': 'User', 'type': 'object'}


In [59]:
from typing import Annotated
from pydantic import BaseModel, Field
from decimal import Decimal

class Product(BaseModel):
    name: Annotated[str, Field(min_length=1, max_length=50)]
    price: Annotated[Decimal, Field(ge=Decimal("0.01"), le=Decimal("1000000.00"), decimal_places=2, max_digits=8)]
    weight: Annotated[Decimal, Field(ge=Decimal("0"), decimal_places=3, max_digits=6)]

# 유효한 데이터
valid_product = Product(name="Laptop", price=Decimal("9999.99"), weight=Decimal("2.500"))
print("Valid Product:")
print(valid_product)
print(valid_product.model_dump())
print(Product.model_json_schema())
print(valid_product.me)

Valid Product:
name='Laptop' price=Decimal('9999.99') weight=Decimal('2.500')
{'name': 'Laptop', 'price': Decimal('9999.99'), 'weight': Decimal('2.500')}
{'properties': {'name': {'maxLength': 50, 'minLength': 1, 'title': 'Name', 'type': 'string'}, 'price': {'anyOf': [{'maximum': 1000000.0, 'minimum': 0.01, 'type': 'number'}, {'type': 'string'}], 'title': 'Price'}, 'weight': {'anyOf': [{'minimum': 0.0, 'type': 'number'}, {'type': 'string'}], 'title': 'Weight'}}, 'required': ['name', 'price', 'weight'], 'title': 'Product', 'type': 'object'}
